In [35]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [36]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [37]:
tf.keras.backend.clear_session()

In [38]:
subject = 'Paraná - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [39]:
data = pd.read_csv('2003_mo_model_input_PR.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraná - Desemprego,Paraná - value,...,Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Paraná - Produção de Cimento (t),Paraná - PIB - Estadual,Paraná - PIB - Construção Civil,Paraná - PIB - Per Capita,Paraná - PIB - Preços de Mercado,Paraná - IDH,Paraná - Consumo de Cimento (t)
0,2003-1,0.724032,11.520143,1.611498,1.036534,1.772069e+08,7.330309e+06,0.969649,8.284196,0.300568,...,NaN,NaN,NaN,332.705560,2.027147e+08,1.060612e+07,17.589536,1.851788e+08,0.776871,189.804
1,2003-2,0.690297,11.189862,1.450389,0.993449,1.773884e+08,7.335910e+06,0.950783,8.277973,0.301710,...,NaN,NaN,NaN,332.945945,2.029477e+08,1.061597e+07,17.592548,1.852326e+08,0.776975,174.713
2,2003-3,0.669681,10.820792,1.870184,0.973020,1.775699e+08,7.341511e+06,0.938332,8.271750,0.302645,...,NaN,NaN,NaN,335.222130,2.031806e+08,1.062582e+07,17.595559,1.852864e+08,0.777079,180.801
3,2003-4,0.660494,10.417840,1.355287,0.940489,1.777514e+08,7.347112e+06,0.926401,8.265527,0.303413,...,NaN,NaN,NaN,334.785737,2.034136e+08,1.063567e+07,17.598570,1.853401e+08,0.777183,180.172
4,2003-5,0.648337,9.959690,1.585538,0.917493,1.779329e+08,7.352713e+06,0.951683,8.259304,0.304034,...,NaN,NaN,NaN,335.050185,2.036466e+08,1.064551e+07,17.601581,1.853939e+08,0.777287,185.597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.549782,...,12.181137,73.843924,1.884424,616.073841,NaN,NaN,NaN,NaN,NaN,412.177
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.546864,...,12.964720,76.682049,1.965570,614.423079,NaN,NaN,NaN,NaN,NaN,337.056
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.544541,...,13.471053,77.179246,1.983732,613.772150,NaN,NaN,NaN,NaN,NaN,340.018
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.540629,...,13.588937,77.540193,2.011078,614.761140,NaN,NaN,NaN,NaN,NaN,383.477


In [40]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data = input_data.fillna(0.0)
input_data

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraná - Desemprego,Paraná - value,Precipitation (mm/day),...,Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Paraná - Produção de Cimento (t),Paraná - PIB - Estadual,Paraná - PIB - Construção Civil,Paraná - PIB - Per Capita,Paraná - PIB - Preços de Mercado,Paraná - IDH
0,2.723741,4.398348,2.132459,3.890153,-2.042341,-2.389042,3.122582,1.200588,-0.990872,0.000000,...,0.000000,0.000000,0.000000,0.000000,-1.688922,-1.661059,-2.360392,-1.926984,-1.800632,-2.165316
1,2.350880,4.222509,1.592202,3.551840,-2.014760,-2.352139,2.970356,1.186349,-0.972615,0.000000,...,0.000000,0.000000,0.000000,0.000000,-1.684978,-1.644277,-2.312924,-1.907242,-1.780025,-2.131422
2,2.123016,4.026019,2.999926,3.391423,-1.987179,-2.315236,2.869895,1.172110,-0.957666,0.000000,...,0.000000,0.000000,0.000000,0.000000,-1.647627,-1.627495,-2.265457,-1.887500,-1.759417,-2.097529
3,2.021477,3.811492,1.273293,3.135979,-1.959598,-2.278333,2.773628,1.157871,-0.945388,0.000000,...,0.000000,0.000000,0.000000,0.000000,-1.654788,-1.610713,-2.217990,-1.867758,-1.738809,-2.063636
4,1.887113,3.567576,2.045407,2.955412,-1.932017,-2.241431,2.977624,1.143632,-0.935451,0.000000,...,0.000000,0.000000,0.000000,0.000000,-1.650449,-1.593931,-2.170523,-1.848016,-1.718202,-2.029743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-2.010387,-0.572934,-1.311366,0.589021,1.365911,0.389193,-1.749976,-0.826445,0.770388,-0.287994,...,-0.817676,-0.508829,-0.038656,-0.839708,1.208142,1.129314,-0.514291,0.743016,0.899324,0.733248
188,-1.870713,-0.588777,-1.417143,1.043728,1.376610,0.370392,-1.593005,-0.840099,0.821622,0.057496,...,-0.259407,0.063830,-0.112274,-0.692111,1.241242,1.120207,-0.507255,0.729140,0.888262,0.688083
189,-1.806230,-0.612606,-1.597784,1.387010,1.387308,0.351592,-1.351489,-0.853753,0.859766,0.623389,...,0.340124,0.673559,-0.183271,-0.596494,1.313851,1.111101,-0.500218,0.715264,0.877199,0.642919
190,-1.727496,-0.640956,-1.400787,1.815728,1.398006,0.332791,-1.198492,-0.867406,0.897253,1.015996,...,0.799611,1.062879,-0.311174,-0.549266,1.363920,1.101994,-0.493182,0.701389,0.866137,0.597754


In [41]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      185.712
1      170.875
2      204.477
3      173.662
4      162.633
        ...   
235        NaN
236        NaN
237        NaN
238        NaN
239        NaN
Name: Paraná - Consumo de Cimento (t), Length: 240, dtype: float64

In [42]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraná - Desemprego,Paraná - value,Precipitation (mm/day),...,Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Paraná - Produção de Cimento (t),Paraná - PIB - Estadual,Paraná - PIB - Construção Civil,Paraná - PIB - Per Capita,Paraná - PIB - Preços de Mercado,Paraná - IDH
0,2.723741,4.398348,2.132459,3.890153,-2.042341,-2.389042,3.122582,1.200588,-0.990872,0.000000,...,0.000000,0.000000,0.000000,0.000000,-1.688922,-1.661059,-2.360392,-1.926984,-1.800632,-2.165316
1,2.350880,4.222509,1.592202,3.551840,-2.014760,-2.352139,2.970356,1.186349,-0.972615,0.000000,...,0.000000,0.000000,0.000000,0.000000,-1.684978,-1.644277,-2.312924,-1.907242,-1.780025,-2.131422
2,2.123016,4.026019,2.999926,3.391423,-1.987179,-2.315236,2.869895,1.172110,-0.957666,0.000000,...,0.000000,0.000000,0.000000,0.000000,-1.647627,-1.627495,-2.265457,-1.887500,-1.759417,-2.097529
3,2.021477,3.811492,1.273293,3.135979,-1.959598,-2.278333,2.773628,1.157871,-0.945388,0.000000,...,0.000000,0.000000,0.000000,0.000000,-1.654788,-1.610713,-2.217990,-1.867758,-1.738809,-2.063636
4,1.887113,3.567576,2.045407,2.955412,-1.932017,-2.241431,2.977624,1.143632,-0.935451,0.000000,...,0.000000,0.000000,0.000000,0.000000,-1.650449,-1.593931,-2.170523,-1.848016,-1.718202,-2.029743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,-0.214006,-0.607704,0.043807,-1.233012,1.031384,0.819304,-0.883659,0.130907,0.732054,1.883148,...,0.852704,1.597886,0.798804,-0.503151,0.692024,1.202413,-0.413954,0.937022,1.076355,1.563435
158,-0.434717,-0.620523,0.210856,-1.299304,1.042716,0.808136,-0.950771,0.076074,0.695547,1.717848,...,0.127298,0.650239,0.567596,-0.758668,0.722303,1.204670,-0.429516,0.934746,1.074731,1.542097
159,-0.524091,-0.631530,0.107070,-1.248662,1.054049,0.796969,-1.028465,0.021241,0.667742,2.005401,...,-0.464108,-0.194246,0.259572,-0.887508,0.729289,1.206927,-0.445078,0.932469,1.073107,1.520760
160,-0.614500,-0.640320,0.392942,-1.068274,1.065381,0.785801,-1.103668,-0.033592,0.634897,1.324868,...,-1.179784,-1.021107,0.220137,-0.935408,0.709776,1.209183,-0.460640,0.930193,1.071484,1.499423


In [43]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      185.712
1      170.875
2      204.477
3      173.662
4      162.633
        ...   
157    280.736
158    352.332
159    280.856
160    319.397
161    295.932
Name: Paraná - Consumo de Cimento (t), Length: 162, dtype: float64

In [44]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [45]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 22)

In [46]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Paraná - Desemprego,Paraná - value,Precipitation (mm/day),...,Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Paraná - Produção de Cimento (t),Paraná - PIB - Estadual,Paraná - PIB - Construção Civil,Paraná - PIB - Per Capita,Paraná - PIB - Preços de Mercado,Paraná - IDH
126,0.888984,-0.460555,-1.048761,-0.368821,0.651397,0.944085,0.045243,-1.195259,1.373912,-0.222769,...,-1.378164,-1.851376,-0.247068,-1.249267,0.887005,0.826002,0.454561,0.988596,0.979507,0.670392
127,0.954254,-0.440372,-0.649312,-0.328087,0.664707,0.947319,0.061828,-1.109331,1.418215,-0.203333,...,-1.062779,-1.487772,-0.245625,-0.911880,0.875394,0.842968,0.424602,0.981838,0.982236,0.678142
128,1.045217,-0.419247,-0.535369,-0.176031,0.678017,0.950553,0.046225,-1.023403,1.454067,-0.170890,...,-0.570413,-0.856435,-0.179589,-0.788932,0.855534,0.859935,0.394643,0.975081,0.984964,0.685892
129,1.176395,-0.397019,-1.288807,-0.113037,0.691327,0.953786,0.032522,-0.937476,1.490026,0.159122,...,0.063145,-0.160407,-0.220686,-0.631550,0.842489,0.876901,0.364683,0.968323,0.987693,0.693642
130,1.303259,-0.376532,-1.507550,-0.022703,0.704637,0.957020,0.042757,-0.851548,1.513267,-0.027260,...,0.663885,0.477898,-0.244957,-0.707205,0.820774,0.893867,0.334724,0.961566,0.990422,0.701392
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-2.010387,-0.572934,-1.311366,0.589021,1.365911,0.389193,-1.749976,-0.826445,0.770388,-0.287994,...,-0.817676,-0.508829,-0.038656,-0.839708,1.208142,1.129314,-0.514291,0.743016,0.899324,0.733248
188,-1.870713,-0.588777,-1.417143,1.043728,1.376610,0.370392,-1.593005,-0.840099,0.821622,0.057496,...,-0.259407,0.063830,-0.112274,-0.692111,1.241242,1.120207,-0.507255,0.729140,0.888262,0.688083
189,-1.806230,-0.612606,-1.597784,1.387010,1.387308,0.351592,-1.351489,-0.853753,0.859766,0.623389,...,0.340124,0.673559,-0.183271,-0.596494,1.313851,1.111101,-0.500218,0.715264,0.877199,0.642919
190,-1.727496,-0.640956,-1.400787,1.815728,1.398006,0.332791,-1.198492,-0.867406,0.897253,1.015996,...,0.799611,1.062879,-0.311174,-0.549266,1.363920,1.101994,-0.493182,0.701389,0.866137,0.597754


In [47]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 22)

In [48]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [49]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(36, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(180,activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [50]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(20)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [51]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[3194660501, 2540215672, 2710867370, 1787035852, 1182666163, 1788870970, 2558438062, 3024413060, 1232266288, 2994785848, 855091102, 2705767462, 2750800333, 1203900580, 1737697738, 3421992166, 2126477788, 1967380698, 4246139853, 1533999960]


Step: 0 ___________________________________________
val_loss: 424.38641357421875
winner_seed: 3194660501


Step: 1 ___________________________________________
val_loss: 443.6375427246094


Step: 2 ___________________________________________
val_loss: 362.2145080566406
winner_seed: 2710867370


Step: 3 ___________________________________________
val_loss: 503.59906005859375


Step: 4 ___________________________________________
val_loss: 338.85565185546875
winner_seed: 1182666163


Step: 5 ___________________________________________
val_loss: 368.6770324707031


Step: 6 ___________________________________________
val_loss: 492.8917236328125


Step: 7 ___________________________________________
val_loss: 367.7853698730469


Step: 8 ___________________

In [ ]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 2s 122ms/step - loss: 80582.0625 - val_loss: 5013.0659
Epoch 2/10000
4/4 [==============================] - 0s 8ms/step - loss: 4840.9287 - val_loss: 4075.1597
Epoch 3/10000
4/4 [==============================] - 0s 9ms/step - loss: 4707.2432 - val_loss: 3289.8362
Epoch 4/10000
4/4 [==============================] - 0s 9ms/step - loss: 4177.8960 - val_loss: 3361.9482
Epoch 5/10000
4/4 [==============================] - 0s 9ms/step - loss: 4168.4238 - val_loss: 3638.0847
Epoch 6/10000
4/4 [==============================] - 0s 9ms/step - loss: 4228.4541 - val_loss: 3147.9011
Epoch 7/10000
4/4 [==============================] - 0s 8ms/step - loss: 4074.7634 - val_loss: 3233.7747
Epoch 8/10000
4/4 [==============================] - 0s 9ms/step - loss: 4195.4014 - val_loss: 3209.3066
Epoch 9/10000
4/4 [==============================] - 0s 9ms/step - loss: 4175.1143 - val_loss: 3708.0151
Epoch 10/10000
4/4 [==============================] 

4/4 [==============================] - 0s 8ms/step - loss: 963.1493 - val_loss: 904.1591
Epoch 79/10000
4/4 [==============================] - 0s 9ms/step - loss: 1001.3043 - val_loss: 861.6919
Epoch 80/10000
4/4 [==============================] - 0s 9ms/step - loss: 998.5201 - val_loss: 1005.3464
Epoch 81/10000
4/4 [==============================] - 0s 9ms/step - loss: 1037.8572 - val_loss: 689.8170
Epoch 82/10000
4/4 [==============================] - 0s 9ms/step - loss: 942.8504 - val_loss: 1248.4059
Epoch 83/10000
4/4 [==============================] - 0s 9ms/step - loss: 1116.2751 - val_loss: 957.5538
Epoch 84/10000
4/4 [==============================] - 0s 10ms/step - loss: 776.5517 - val_loss: 865.0437
Epoch 85/10000
4/4 [==============================] - 0s 9ms/step - loss: 909.5887 - val_loss: 1141.0659
Epoch 86/10000
4/4 [==============================] - 0s 10ms/step - loss: 938.8246 - val_loss: 762.6562
Epoch 87/10000
4/4 [==============================] - 0s 9ms/step - los

Epoch 156/10000
4/4 [==============================] - 0s 9ms/step - loss: 628.4887 - val_loss: 438.0019
Epoch 157/10000
4/4 [==============================] - 0s 9ms/step - loss: 599.0984 - val_loss: 487.2386
Epoch 158/10000
4/4 [==============================] - 0s 10ms/step - loss: 761.6763 - val_loss: 403.7064
Epoch 159/10000
4/4 [==============================] - 0s 10ms/step - loss: 663.6108 - val_loss: 999.8555
Epoch 160/10000
4/4 [==============================] - 0s 10ms/step - loss: 817.7205 - val_loss: 940.3138
Epoch 161/10000
4/4 [==============================] - 0s 9ms/step - loss: 813.9246 - val_loss: 538.2367
Epoch 162/10000
4/4 [==============================] - 0s 9ms/step - loss: 732.8977 - val_loss: 728.0884
Epoch 163/10000
4/4 [==============================] - 0s 10ms/step - loss: 800.6381 - val_loss: 1734.1621
Epoch 164/10000
4/4 [==============================] - 0s 10ms/step - loss: 1232.1013 - val_loss: 838.7564
Epoch 165/10000
4/4 [===========================

4/4 [==============================] - 0s 9ms/step - loss: 510.0572 - val_loss: 758.3552
Epoch 234/10000
4/4 [==============================] - 0s 9ms/step - loss: 526.0228 - val_loss: 1074.6614
Epoch 235/10000
4/4 [==============================] - 0s 9ms/step - loss: 664.6710 - val_loss: 650.6874
Epoch 236/10000
4/4 [==============================] - 0s 9ms/step - loss: 471.4243 - val_loss: 574.2076
Epoch 237/10000
4/4 [==============================] - 0s 10ms/step - loss: 467.1141 - val_loss: 731.9100
Epoch 238/10000
4/4 [==============================] - 0s 10ms/step - loss: 771.8764 - val_loss: 757.1384
Epoch 239/10000
4/4 [==============================] - 0s 9ms/step - loss: 765.3004 - val_loss: 525.8508
Epoch 240/10000
4/4 [==============================] - 0s 10ms/step - loss: 439.4743 - val_loss: 725.1304
Epoch 241/10000
4/4 [==============================] - 0s 10ms/step - loss: 621.0557 - val_loss: 640.3925
Epoch 242/10000
4/4 [==============================] - 0s 9ms/step

Epoch 311/10000
4/4 [==============================] - 0s 9ms/step - loss: 521.7089 - val_loss: 603.4453
Epoch 312/10000
4/4 [==============================] - 0s 9ms/step - loss: 683.1398 - val_loss: 839.5533
Epoch 313/10000
4/4 [==============================] - 0s 9ms/step - loss: 575.0963 - val_loss: 447.9884
Epoch 314/10000
4/4 [==============================] - 0s 9ms/step - loss: 525.7447 - val_loss: 434.1640
Epoch 315/10000
4/4 [==============================] - 0s 9ms/step - loss: 413.0940 - val_loss: 661.5126
Epoch 316/10000
4/4 [==============================] - 0s 9ms/step - loss: 459.1537 - val_loss: 642.6840
Epoch 317/10000
4/4 [==============================] - 0s 11ms/step - loss: 405.3203 - val_loss: 504.8061
Epoch 318/10000
4/4 [==============================] - 0s 10ms/step - loss: 361.3445 - val_loss: 455.7154
Epoch 319/10000
4/4 [==============================] - 0s 9ms/step - loss: 389.6881 - val_loss: 419.9150
Epoch 320/10000
4/4 [==============================] 

...

Epoch 1475/10000
4/4 [==============================] - 0s 10ms/step - loss: 102.8797 - val_loss: 375.5581
Epoch 1476/10000
4/4 [==============================] - 0s 9ms/step - loss: 80.2983 - val_loss: 394.9596
Epoch 1477/10000
4/4 [==============================] - 0s 9ms/step - loss: 83.2001 - val_loss: 380.8395
Epoch 1478/10000
4/4 [==============================] - 0s 9ms/step - loss: 73.6601 - val_loss: 370.8687
Epoch 1479/10000
4/4 [==============================] - 0s 9ms/step - loss: 76.1416 - val_loss: 379.4164
Epoch 1480/10000
4/4 [==============================] - 0s 9ms/step - loss: 60.9301 - val_loss: 402.5115
Epoch 1481/10000
4/4 [==============================] - 0s 9ms/step - loss: 65.2231 - val_loss: 387.6917
Epoch 1482/10000
4/4 [==============================] - 0s 9ms/step - loss: 75.9089 - val_loss: 407.5648
Epoch 1483/10000
4/4 [==============================] - 0s 9ms/step - loss: 67.2543 - val_loss: 393.6885
Epoch 1484/10000
4/4 [==============================]

Epoch 1553/10000
4/4 [==============================] - 0s 8ms/step - loss: 91.5489 - val_loss: 379.6647
Epoch 1554/10000
4/4 [==============================] - 0s 8ms/step - loss: 51.4930 - val_loss: 398.2099
Epoch 1555/10000
4/4 [==============================] - 0s 8ms/step - loss: 78.8840 - val_loss: 438.2058
Epoch 1556/10000
4/4 [==============================] - 0s 8ms/step - loss: 68.9301 - val_loss: 381.4283
Epoch 1557/10000
4/4 [==============================] - 0s 8ms/step - loss: 88.3013 - val_loss: 400.0948
Epoch 1558/10000
4/4 [==============================] - 0s 9ms/step - loss: 87.4096 - val_loss: 464.1735
Epoch 1559/10000
4/4 [==============================] - 0s 10ms/step - loss: 65.7226 - val_loss: 381.1062
Epoch 1560/10000
4/4 [==============================] - 0s 11ms/step - loss: 70.3323 - val_loss: 438.7005
Epoch 1561/10000
4/4 [==============================] - 0s 10ms/step - loss: 72.5964 - val_loss: 377.2935
Epoch 1562/10000
4/4 [==============================

4/4 [==============================] - 0s 9ms/step - loss: 84.8382 - val_loss: 393.5322
Epoch 1631/10000
4/4 [==============================] - 0s 9ms/step - loss: 72.0653 - val_loss: 605.9165
Epoch 1632/10000
4/4 [==============================] - 0s 9ms/step - loss: 186.8090 - val_loss: 491.3959
Epoch 1633/10000
4/4 [==============================] - 0s 8ms/step - loss: 155.9715 - val_loss: 426.8441
Epoch 1634/10000
4/4 [==============================] - 0s 8ms/step - loss: 81.3770 - val_loss: 505.0677
Epoch 1635/10000
4/4 [==============================] - 0s 8ms/step - loss: 123.6508 - val_loss: 455.4818
Epoch 1636/10000
4/4 [==============================] - 0s 8ms/step - loss: 69.0553 - val_loss: 442.4112
Epoch 1637/10000
4/4 [==============================] - 0s 9ms/step - loss: 84.2952 - val_loss: 400.4150
Epoch 1638/10000
4/4 [==============================] - 0s 9ms/step - loss: 132.9895 - val_loss: 420.8581
Epoch 1639/10000
4/4 [==============================] - 0s 9ms/step 

4/4 [==============================] - 0s 9ms/step - loss: 54.4630 - val_loss: 431.2797
Epoch 1709/10000
4/4 [==============================] - 0s 9ms/step - loss: 66.7766 - val_loss: 377.6790
Epoch 1710/10000
4/4 [==============================] - 0s 9ms/step - loss: 87.0449 - val_loss: 506.1039
Epoch 1711/10000
4/4 [==============================] - 0s 9ms/step - loss: 88.0419 - val_loss: 453.0173
Epoch 1712/10000
4/4 [==============================] - 0s 9ms/step - loss: 77.6313 - val_loss: 401.4249
Epoch 1713/10000
4/4 [==============================] - 0s 8ms/step - loss: 53.4079 - val_loss: 384.2848
Epoch 1714/10000
4/4 [==============================] - 0s 9ms/step - loss: 50.8016 - val_loss: 374.2750
Epoch 1715/10000
4/4 [==============================] - 0s 11ms/step - loss: 64.5201 - val_loss: 372.6832
Epoch 1716/10000
4/4 [==============================] - 0s 9ms/step - loss: 53.1966 - val_loss: 380.6253
Epoch 1717/10000
4/4 [==============================] - 0s 9ms/step - l

Epoch 1786/10000
4/4 [==============================] - 0s 8ms/step - loss: 74.7888 - val_loss: 418.0433
Epoch 1787/10000
4/4 [==============================] - 0s 8ms/step - loss: 64.5933 - val_loss: 429.5572
Epoch 1788/10000
4/4 [==============================] - 0s 9ms/step - loss: 59.2239 - val_loss: 382.0632
Epoch 1789/10000
4/4 [==============================] - 0s 9ms/step - loss: 56.8011 - val_loss: 452.1792
Epoch 1790/10000
4/4 [==============================] - 0s 9ms/step - loss: 74.1073 - val_loss: 333.8448
Epoch 1791/10000
4/4 [==============================] - 0s 9ms/step - loss: 59.6329 - val_loss: 430.3805
Epoch 1792/10000
4/4 [==============================] - 0s 9ms/step - loss: 76.2460 - val_loss: 409.1441
Epoch 1793/10000
4/4 [==============================] - 0s 8ms/step - loss: 44.9478 - val_loss: 418.0812
Epoch 1794/10000
4/4 [==============================] - 0s 8ms/step - loss: 40.9269 - val_loss: 422.6132
Epoch 1795/10000
4/4 [==============================] -

In [ ]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [ ]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

In [ ]:
display(mae)
display(mape)

In [ ]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [ ]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)